# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [ ]:
#adds api_key file path back into directories that pandas searches through to find api_keys

import sys
sys.path.append("..")

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,whitehorse,60.7161,-135.0538,276.58,57,75,5.14,CA,1683628533
1,1,iqaluit,63.7506,-68.5145,265.00,78,20,4.63,CA,1683628534
2,2,grytviken,-54.2811,-36.5092,273.80,93,84,0.18,GS,1683628534
3,3,ancud,-41.8697,-73.8203,282.21,88,96,1.21,CL,1683628371
4,4,labytnangi,66.6572,66.4183,274.36,100,100,6.50,RU,1683628534


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
city_plot = city_data_df.hvplot.points(
    "Lat",
    "Lng",
    geo =True,
    size = "Humidity",
    scale = 0.01,
    frame_width = 700,
    frame_height = 500
    
)

# Display the map
city_plot

:Points   [Lat,Lng]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [32]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df= city_data_df.loc[city_data_df["Max Temp"].between(294,300),:]

city_data_df= city_data_df.loc[city_data_df["Humidity"].between(30,50),:]
# Drop any rows with null values
city_data_df = city_data_df.dropna(subset=['Country'])
# Display sample data
city_data_df



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
44,44,bredasdorp,-34.5322,20.0403,295.33,48,59,1.32,ZA,1683628542
65,65,cairo,30.0626,31.2497,298.57,41,40,2.57,EG,1683628378
101,101,masvingo,-20.0637,30.8277,298.47,34,5,3.51,ZW,1683628552
125,125,palapye,-22.5460,27.1251,299.58,37,23,3.44,BW,1683628556
169,169,la puebla de montalban,39.8630,-4.3592,299.00,36,91,1.96,ES,1683628565
200,200,chivhu,-19.0211,30.8922,297.19,30,4,3.45,ZW,1683628572
205,205,kununurra,-15.7667,128.7333,295.14,43,57,2.06,AU,1683628573
235,235,maloye ganyushkino,46.6000,49.2667,294.02,37,76,8.26,KZ,1683628579
272,272,bayganin,48.6947,55.8742,297.83,31,99,7.20,KZ,1683628586
287,287,villa del campo,40.1422,-6.4268,295.96,40,100,5.40,ES,1683628589


### Step 3: Create a new DataFrame called `hotel_df`.

In [33]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
44,44,bredasdorp,-34.5322,20.0403,295.33,48,59,1.32,ZA,1683628542,
65,65,cairo,30.0626,31.2497,298.57,41,40,2.57,EG,1683628378,
101,101,masvingo,-20.0637,30.8277,298.47,34,5,3.51,ZW,1683628552,
125,125,palapye,-22.5460,27.1251,299.58,37,23,3.44,BW,1683628556,
169,169,la puebla de montalban,39.8630,-4.3592,299.00,36,91,1.96,ES,1683628565,
200,200,chivhu,-19.0211,30.8922,297.19,30,4,3.45,ZW,1683628572,
205,205,kununurra,-15.7667,128.7333,295.14,43,57,2.06,AU,1683628573,
235,235,maloye ganyushkino,46.6000,49.2667,294.02,37,76,8.26,KZ,1683628579,
272,272,bayganin,48.6947,55.8742,297.83,31,99,7.20,KZ,1683628586,
287,287,villa del campo,40.1422,-6.4268,295.96,40,100,5.40,ES,1683628589,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = # YOUR CODE HERE
params = {
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

SyntaxError: invalid syntax (788381735.py, line 2)

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE